In [ ]:
%cd ./drive/MyDrive/Colab\ Notebooks

import torch 
import numpy as np 
from einops import rearrange
from torch.nn import functional as F
from torch import nn



/content/drive/MyDrive/Colab Notebooks


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self , heads , embedding_dim , sequence_size , batch_dim ):
        super(SelfAttention ,self).__init__() 
        self.heads = heads 
        self.embedding_dim = embedding_dim 
        self.sequence_size = sequence_size
        self.batch_dim = batch_dim

        self.transform_to_keys = nn.Linear(embedding_dim , embedding_dim * heads)
        self.transform_to_queries = nn.Linear(embedding_dim , embedding_dim*heads)
        self.transform_to_values = nn.Linear(embedding_dim , embedding_dim * heads)

        self.final = nn.Linear(heads*embedding_dim  , embedding_dim)

        def forward(self , input ,  mask = None):
            assert input.shape == (self.batch_dim ,self.sequence_size , self.embedding_dim) , "Input shape is not (b x s x e )"
            queries = einops.rearrange(self.transform_to_queries(input) , \
                                       'batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            keys = einops.rearrange(self.transform_to_keys(input) , \
                                   ' batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            values = einops.rearrange(self.transform_to_values(input) ,\
                                      'batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            dot_product = torch.einsum("bse,bte->bst" , [keys ,queries]) / np.sqrt(self.embedding_dim)
            if mask is not None :
                dot_product = dot_product.masked_fill(mask[None] , -np.inf)
            attention_weights = torch.nn.functional.softmax(dot_product , dim =-1)
            attention_applied_output = torch.einsum('bst,bte->bse',[attention_weights , values]) # b here actually is b*h 

            return self.final(einops.rearrange(attention_applied_output ,\
                                               "(batch heads) seq_size emb_size ->  batch seq_size (heads emb_dim)" , heads = self.heads))




In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self  ,ffnn_dim , heads , embedding_dim , sequence_size , batch_dim):
        super(TransformerBlock , self).__init__() 
        self.heads = heads 
        self.embedding_dim = embedding_dim 
        self.sequence_size = sequence_size
        self.batch_dim = batch_dim

        self.norm1 = nn.LayerNorm()
        self.norm2 = nn.LayerNorm()

        self.MHA = MultiHeadAttention(self.heads , self.embedding_dim , self.sequence_size ,self.batch_dim)
        self.drop = nn.Dropout(0.3)
        self.ffnn = nn.Sequential(
            nn.Linear(embedding_dim , ffnn_dim) ,nn.LeakyReLU(0.3) , nn.Dropout(dropout) , 
            nn.Linear(ffnn ,embedding_dim) ,nn.LeakyReLU(0.3) , nn.Dropout(dropout) 
        )
        def forward(self , input , mask = None):
            output = self.norm1(self.drop(self.MHA(input , mask )) + input)
            return self.norm2(self.ffnn(output) + output)


In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self ,n_blocks , ffnn_dim , heads , embedding_dim , sequence_size , batch_dim)):
        super(Transformer , self).__init__()
        self.blocks = [TransformerBlock(ffnn_dim , heads , embedding_dim , sequence_size , batch_dim) for _ in range(n_blocks))]
        self.layers = nn.Sequential(self.blocks)

        def forward(self , input , mask) = None:
            return self.layers(input)

            


In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self , image_dim   ,batch_dim ,num_classes =  10, channels_in=3 , p_dim = 512, patch_size=16 , \
                  blocks=6 , heads=4 , linear_block_dim=1024,classification = True  , \
                 drop = 0.2  , transformer_enc = None):
        """
        Parameters .

        image_dim: image_size C x H x W
        num_classes: classification task classes
        channels_in: no of channels of image input (C)
        batch_dim : for Transformer encoder
        p_dim: the linear layer's dim to project the patches for Multi Head Self Attention
        patch_size: patch dim to divide image in
        blocks: no of transformer encoder blocks
        heads: not of Multi Head Attention Heads
        linear_block_dim: transformer's linear block's dim
        classification : for an extra CLS token
        drop : probobility to drop in dropout layers
        transformer_enc  : transformer encoder block provided
        
        """

        super(VisionTransformer,self)__init__()
        assert image_dim % patch_size == 0 ,f"Patch size {patch_size} is not divisible."

        self.patch_dim = patched_dim
        self.patch_size = patch_size
        self.blocks = blocks
        self.classify = classification
        N_patches = (image_dim // self.patch_size)**2
        self.token_in = in_channels *((self.patch_size)**2) # P2C -->Linear projection--> p_dim
        self.p_dim = p_dim

        self.Project_Patches = nn.Linear(self.token_in , self.p_dim)
        self.Embedding_dropout = nn.Dropout(drop)

        if self.classify:
            self.cls = nn.parameter(torch.randn(1,1,self.p_dim))
            self.positinoal_embeddings = nn.Parameter(torch.randn(tokens+1 , self.p_dim))
            self.top_head = nn.Linear(self.p_dim ,num_classes)
        else : 
            self.positinoal_embeddings = nn.Parameter(torch.randn(tokens , self.p_dim))


        if transformer_enc is None:
            self.Transformer_encoder = TransformerEncoder(blocks , p_dim , heads , linear_block_dim , N_patches  ,batch_dim)


    `   else :
            self.Transformer_encoder = transformer_enc

    def expand_cls_to_batch(self, batch):
        """
        Parameters:
            batch: batch size
        Returns: 
            cls token expanded to the batch size
        """
        return self.cls_token.expand([batch, -1, -1])


    def forward(self, img, mask=None):
        batch_size = img.shape[0]
        img_patches = rearrange(
            img, 'b c (p_1 x) (p_2 y) -> b (x y) (p_1 p_2 c)',
                                p_1=self.patch_size, p_2=self.patch_size)
        
        # Linear projection of patches with linear layer
        #  +  add pos emb

        img_patches = self.Project_Patches(img_patches)
        if self.classification:
            img_patches = torch.cat(
                (self.expand_cls_to_batch(batch_size), img_patches), dim=1)
        patch_embeddings = self.Embedding_dropout(img_patches + self.positional_embeddings)

        # feed patch_embeddings and output of transformer , shape: [batch, tokens, dim]

        y = self.Transformer_encoder(patch_embeddings, mask)
        if self.classification:
            # we index only the cls token for classification
            return self.top_head(y[:, 0, :])
        else:
            return y